<div dir="rtl">

## م2.5 — إنشاء تحليل بيانات مناخية قابل لإعادة الإنتاج

*جزء من:* <span dir="ltr">[**Computational Climate Science**](https://github.com/OpenClimateScience/M2-Computational-Climate-Science)</span> | **الدرس السابق** | **الدرس التالي**

**المحتويات:**

- [إنشاء سير عمل قابل لإعادة الإنتاج](#Creating-a-reproducible-workflow)
  - [سير عملنا: تنزيل البيانات (الخطوة 1)](#Our-workflow:-Downloading-the-data-(Step-1))
  - [سير عملنا: معالجة البيانات (الخطوة 2)](#Our-workflow:-Data-processing-(Step-2))
- [ملفات مشروع قابل لإعادة الإنتاج](#A-reproducible-project's-files)
- [مقارنة عدة سنوات من بيانات المناخ](#Comparing-multiple-years-of-climate-data)
  - [حساب إشعاع أعلى الغلاف الجوي (TOA)](#Computing-TOA-radiation)
  - [حساب التبخر-نتح الكامن (PET)](#Computing-PET)
  - [إعادة أخذ عينات بيانات CHIRPS](#Resampling-the-CHIRPS-data)
  - [اختيار جزء من سلسلة زمنية في xarray](#Selecting-part-of-an-xarray-time-series)

</div>


<div dir="rtl">

---

## إنشاء سير عمل قابل لإعادة الإنتاج

في الدرس السابق، استخدمنا <span dir="ltr">`dask`</span> و<span dir="ltr">`xarray`</span> لقراءة مجموعة من ملفات <span dir="ltr">netCDF</span>، ثم قمنا **بتمرير** دالة **متجهة** فوق كل مصفوفة ضمن سلسلة زمنية. وأنتجنا رسمًا بيانيًا يُظهر نسبة **الهطول إلى PET** لمدينة تيارت، الجزائر، في الأشهر الأولى من عام <span dir="ltr">2024</span> خلال جفاف شديد.

بحد ذاته، الرسم البياني لا يخبرنا مدى شدة الجفاف في تيارت. رغم أن الهطول في المنطقة عوّض أقل من <span dir="ltr">5%</span> من المياه المفقودة خلال الأشهر الماضية، فقد يكون ذلك جزءًا من الدورة الموسمية الطبيعية. في الواقع، نعلم أن الفترة من يناير إلى أبريل تُعد فترة أكثر رطوبة نسبيًا في تيارت، لكن يبقى السؤال: **هل يمكننا مقارنة هذا العام بسنوات سابقة؟**

كلما أردنا تطبيق تحليل مكتمل على مجموعة بيانات جديدة — عبر الزمن أو عبر المكان — فهذه فرصة لتحسين كيفية تمثيل سير العمل لدينا. انظر إلى السكربتين أدناه، فهما يمثلان أول خطوتين في سير عملنا.

</div>


<div dir="rtl">

### سير عملنا: تنزيل البيانات (الخطوة 1)

قد يُسمّى الملف الأول شيئًا مثل **<span dir="ltr">`YYYYMMDD_step1_download_MERRA2_data.py`</span>**. تذكّر أن <span dir="ltr">`YYYYMMDD`</span> هو تاريخ اليوم، وهو يساعدنا على **ربط ملفات المخرجات بالشيفرة التي أنشأتها.**

لاحظ أن ملفات بايثون ذات الامتداد <span dir="ltr">`*.py`</span> يمكن أن تحتوي على **سلسلة توثيق على مستوى الملف (file-level docstring)**. في المثال أدناه، هي سلسلة بايثون متعددة الأسطر التي تبدأ بـ <span dir="ltr">`'''`</span>. ويجب أن تبدأ سلسلة توثيق مستوى الملف من السطر الأول في الملف. وهي شديدة الأهمية لعمليات إعادة الإنتاج؛ لأن أول سطر في الملف هو أول مكان ستنظر إليه لفهم هدف الملف!

```python
'''
Downloads MERRA-2 M2SDNXSLV data, for the first 5 months of a year.
Read more about MERRA-2 here:

    https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/

Data are downloaded to this folder:

    data_raw/MERRA2
'''

import earthaccess

DATA_YEAR = 2023

auth = earthaccess.login()

results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = (f"{DATA_YEAR}-01-01", f"{DATA_YEAR}-05-31"))

# Could take about 1 minute on a broadband connection
earthaccess.download(results, 'data_raw/MERRA2')
```

</div>


<div dir="rtl">

&#x1F449; **من الأعلى إلى الأسفل، لاحظ ما يلي:**

- لدينا **سلسلة توثيق على مستوى الملف** في أعلى السكربت تحتوي معلومات مهمة عن هدف السكربت، وأين يمكن الحصول على معلومات إضافية، وكيف يغيّر السكربت بنية الملفات.
- كل أوامر <span dir="ltr">`import`</span> موضوعة قرب أعلى السكربت. هذا يُشير للقارئ ما هي الوحدات المطلوبة لتشغيل السكربت. لا نريد وضع <span dir="ltr">`import`</span> في منتصف/آخر السكربت لأن ذلك يجعل العثور عليها أصعب، وقد يؤدي إلى خطأ مفاجئ من نوع <span dir="ltr">`ImportError`</span> عند التشغيل.
- المعاملات التي قد نغيّرها عند إعادة تشغيل السكربت موضّحة بوضوح باستخدام أحرف كبيرة وتعريفها قرب الأعلى. مثلًا، <span dir="ltr">`DATA_YEAR`</span> متغيّر قد نريد تغييره عند تشغيل السكربت لسنوات مختلفة. وضعه في الأعلى وبأحرف كبيرة يقلّل عناء البحث داخل السكربت عن الجزء الذي يجب تغييره.

</div>


<div dir="rtl">

### سير عملنا: معالجة البيانات (الخطوة 2)

الخطوة التالية هي قراءة ملفات البيانات وحساب إشعاع أعلى الغلاف الجوي <span dir="ltr">(TOA)</span>. وقد يُسمّى الملف الثاني **<span dir="ltr">`YYYYMMDD_step2_compute_TOA_radiation.py`</span>**.

```python
'''
Computes top-of-atmosphere (TOA) radiation from a series of MERRA-2 
M2SDNXSLV files, then writes an output netCDF file. TOA radiation is
calculated according to the FAO formula for extraterrestrial radiation:

    https://www.fao.org/4/X0490E/x0490e07.htm#radiation
'''

import numpy as np
import xarray as xr

# NOTE: This will be different on your computer system and you should
#   use an absolute path, not a relative path
MERRA2_DATA_DIR = './data_raw/MERRA2'
DATA_YEAR = 2023
OUTPUT_FILE = f'./outputs/YYYYMMDD_MERRA2_{DATA_YEAR}_with_TOA-radiation.nc'

def main():
    ds = xr.open_mfdataset(f'{MERRA2_DATA_DIR}/*{DATA_YEAR}*.nc4', chunks = 'auto')
    lats = ds['lat'].values.reshape((361, 1, 1))\
        .repeat(ds.lon.size, axis = 1)\
        .repeat(ds.time.size, axis = 2)
    ds['lat_grid'] = (('lat', 'lon', 'time'), lats)

    # Compute TOA radiation
    template = ds['T2MMEAN']
    template.name = 'toa_radiation'
    result = xr.map_blocks(toa_radiation_wrapper, ds, template = template)
    toa_result = result.compute()
    # Converting TOA Radiation from [MJ m-2 day-1] to [mm H2O day-1]
    ds['toa_radiation'] = toa_result * 0.408
    
    # Write the file to disk, just the important variables
    ds = ds[['T2MMAX', 'T2MMEAN', 'T2MMIN', 'toa_radiation']]
    comp = dict(zlib = True, complevel = 5)
    encoding = {var: comp for var in ds.data_vars}
    ds.to_netcdf(OUTPUT_FILE, format = 'NETCDF4', encoding = encoding)

    
def toa_radiation(latitude, doy):
    '''
    Top-of-atmosphere (TOA) radiation for a given latitude (L) and day of year
    (DOY) can be calculated as:

    R = ((24 * 60) / pi) * G * d * (w * sin(L) * sin(D) + cos(L) * cos(D) * sin(w))

    Where G is the solar constant, 0.0820 [MJ m-2 day-1]; d is the earth-sun
    distance; w is the sunset hour angle; and D is the solar declination angle.
    
    For more information, consult the FAO documentation:

        https://www.fao.org/4/X0490E/x0490e07.htm#radiation
    
    Parameters
    ----------
    latitude : float
        The latitude on earth, in degrees
    doy : int
        The day of the year (DOY), an integer on [1,366]
    
    Returns
    -------
    Number
        Top-of-atmosphere (TOA) radiation, in [MJ m-2 day-1]
    '''
    solar_constant = 0.0820 # [MJ m-2 day-1]
    pi = 3.14159
    
    # Convert latitude from degrees to radians
    lat_radians = np.deg2rad(latitude)
    # Earth-Sun distance, as a function of day-of-year (DOY)
    earth_sun_dist = 1 + 0.0033 * np.cos(doy * ((2 * pi) / 365))
    # Solar declination, as a function of DOY
    declination = 0.409 * np.sin(doy * ((2 * pi) / 365) - 1.39)
    
    # Sunset hour angle; we use np.where() below to guard against
    #   warnings where arccos() would return invalid values, which
    #   happens when the argument is outside [-1, 1]
    _hour_angle = -np.tan(lat_radians) * np.tan(declination)
    _hour_angle = np.where(np.abs(_hour_angle) > 1, np.nan, _hour_angle)
    sunset_hour_angle = np.arccos(_hour_angle)
    
    return ((24 * 60) / pi) * solar_constant * earth_sun_dist *\
        (sunset_hour_angle * np.sin(lat_radians) * np.sin(declination) +
            np.cos(lat_radians) * np.cos(declination) * np.sin(sunset_hour_angle))


def toa_radiation_wrapper(dataset):
    '''
    Wraps toa_radiation to work with an xarray.Dataset

    Parameters
    ----------
    dataset : xarray.Dataset
        Input dataset with "lat_grid" and "time" variables

    Returns
    -------
    xarray.DataArray
    '''
    return toa_radiation(dataset['lat_grid'], dataset['time.dayofyear'])


# If the file is run as a script, run the main() function
if __name__ == '__main__':
    main()
```

</div>


<div dir="rtl">

&#x1F449; **مرة أخرى، لاحظ ما يلي:**

- وجود **سلسلة توثيق على مستوى الملف** وأوامر <span dir="ltr">`import`</span> قرب الأعلى يساعد المستخدمين على فهم هدف السكربت ومعرفة الوحدات المطلوبة لتشغيله.
- الدالة <span dir="ltr">`toa_radiation()`</span> تحتوي أيضًا على **سلسلة توثيق على مستوى الدالة (function-level docstring)** تشرح: كيف تعمل الدالة، وما هي **مدخلاتها**، وما هو **الناتج**.

&#x1F449; **انظر إلى السطر الذي يحتوي <span dir="ltr">`if __name__ == '__main__'`</span>؛ ماذا يعني ذلك؟**

- كل ملف بايثون (<span dir="ltr">`*.py`</span>) يمكن تشغيله بطريقتين: إما بتنفيذه كسكربت عبر <span dir="ltr">`python myscript.py`</span> أو عبر *استيراده* كوحدة (module) مثل <span dir="ltr">`import myscript`</span>.
- عند تشغيل كود بايثون في ملف، يتم إنشاء متغير باسم <span dir="ltr">`__name__`</span> يدل على اسم الوحدة. إذا تم تشغيل الملف كسكربت مباشرة، فإن قيمة <span dir="ltr">`__name__`</span> تصبح <span dir="ltr">`'__main__'`</span>. لذلك يمكننا استخدام <span dir="ltr">`__name__`</span> لنعرف هل الكود يُنفّذ كسكربت أم أنه مستورد كوحدة.

**لماذا يهمّنا هذا الفرق؟** لأن استيراد ملف كوحدة يؤدي إلى تنفيذ كل الكود الموجود فيه. وبالتالي، أي كود خارج **تعريف الدوال** سيتم تنفيذه عند كل عملية استيراد — وهذا غالبًا ليس ما نريده، خاصة إذا كان الملف يحتوي دوال مفيدة مثل <span dir="ltr">`toa_radiation()`</span> نريد **إعادة استخدامها** في مكان آخر، مثل: <span dir="ltr">`from myscript import toa_radiation()`</span>.

إذن، الكود الذي نريد تشغيله *فقط عندما يُشغَّل الملف كسكربت* يجب وضعه داخل دالة مثل <span dir="ltr">`main()`</span>، ثم يتم استدعاؤها بشكل شرطي:
```python
# If the file is run as a script, run the main() function
if __name__ == '__main__':
    main()
```

إذا كان هذا مربكًا الآن، فاعتبر هذا الشرط نوعًا من "الحيلة" في بايثون التي تسمح لنا بكتابة ملفات يمكن تشغيلها كسكربت أو استيرادها كوحدة.

</div>


<div dir="rtl">

---

## ملفات مشروع قابل لإعادة الإنتاج

هناك خطوات أخرى في التحليل، لكن نترك لك كتمرين كتابة سكربتات قابلة لإعادة الاستخدام وموثّقة جيدًا مثل المثال أعلاه. قد يبدو مجلد المشروع النهائي لديك مثل هذا:

![](./assets/M2_file_tree_workflow.png)

&#x1F449; **لاحظ أن كل ملفات <span dir="ltr">`scripts`</span> تتضمن كلمات مثل <span dir="ltr">`step1`</span> أو <span dir="ltr">`step2`</span> أو <span dir="ltr">`step3`</span> لتوضيح ترتيب التنفيذ.** هذا يساعد أي شخص (وقد يساعدك أنت لاحقًا) لمعرفة ترتيب تشغيل سير العمل، خصوصًا إذا عدت للمشروع بعد أشهر ولم تتذكر ما فعلته سابقًا!

</div>


<div dir="rtl">

---

## مقارنة عدة سنوات من بيانات المناخ

في النهاية، نريد الإجابة عن سؤالنا من الدرس السابق: كيف تقارن نسبة **الهطول إلى PET** في شتاء <span dir="ltr">2024</span> مع شتاء <span dir="ltr">2023</span> في تيارت؟

سير العمل القابل لإعادة الإنتاج أعلاه يمكنه الإجابة عن هذا السؤال. دعنا أيضًا نرى كيف نجيب عنه في **جلسة بايثون تفاعلية**، أي داخل هذا الدفتر <span dir="ltr">(Jupyter Notebook)</span>. إذا كنا قد نزّلنا بيانات <span dir="ltr">MERRA-2</span> لكل من <span dir="ltr">2023</span> و<span dir="ltr">2024</span>، فنحن جاهزون لتحميل البيانات باستخدام <span dir="ltr">`xarray.open_mfdataset()`</span>.

</div>


In [ ]:
import xarray as xr
from matplotlib import pyplot

ds_chirps = xr.open_mfdataset('data_raw/CHIRPS/CHIRPS-v2_Africa_monthly_2014-2024.nc')
ds_merra2 = xr.open_mfdataset('data_raw/MERRA2/*.nc4', chunks = 'auto')

<div dir="rtl">

سكربت بايثون لدينا <span dir="ltr">`YYYYMMDD_step2_compute_TOA_radiation.py`</span> يحتوي دالة مفيدة وقابلة لإعادة الاستخدام اسمها <span dir="ltr">`toa_radiation()`</span>. **كيف نستفيد من هذه الدالة دون نسخ ولصق تعريف الدالة في سكربت آخر أو داخل هذا الدفتر؟** بشكل عام، يجب أن نتجنب وجود أكثر من نسخة لأي دالة. أحد أهم أسباب كتابة الدوال هو تجنب كتابة نفس الشيفرة مرتين!

&#x1F449; كما ذكرنا سابقًا، عندما نضع الشرط <span dir="ltr">`if __name__ == '__main__':`</span> في السكربت، يصبح من السهل استيراد السكربت كوحدة دون تنفيذ جزء التشغيل الرئيسي تلقائيًا. ما دام بإمكاننا رؤية السكربت <span dir="ltr">`YYYYMMDD_step2_compute_TOA_radiation.py`</span> ضمن شجرة الملفات، فيجب أن نستطيع استيراده كوحدة. أدناه، نستخدم مكتبة <span dir="ltr">`os`</span> لعرض ملفات المجلد الحالي. يمكننا رؤية وجود مجلد <span dir="ltr">`scripts`</span> داخل هذا المجلد...

</div>


In [ ]:
import os

# Display the files in the current directory
files = os.listdir('.') # The single dot "." indicates the current directory
files.sort()
files

<div dir="rtl">

وداخل <span dir="ltr">`scripts`</span> يوجد سكربت بايثون الذي نريد استيراده كوحدة!

</div>


In [ ]:
os.listdir('scripts')

<div dir="rtl">

كيف نفعل ذلك؟ لأن كِلا المجلدات وملفات بايثون <span dir="ltr">`*.py`</span> يمكن التعامل معها كوحدات (modules)، فكل ما نحتاجه هو كتابة أمر استيراد كوحدة مثل المثال أدناه.

</div>


In [ ]:
from scripts.YYYYMMDD_step2_compute_TOA_radiation import toa_radiation_wrapper

<div dir="rtl">

لاحظ أننا استوردنا اسمًا واحدًا فقط من هذا السكربت: دالة <span dir="ltr">`toa_radiation_wrapper()`</span>، والتي تمنحنا الوصول إلى دالة <span dir="ltr">`toa_radiation()`</span>. وبما أننا كتبنا **سلسلة توثيق ممتازة على مستوى الدالة** سابقًا، يمكننا رؤية كل تفاصيل الاستخدام باستدعاء <span dir="ltr">`help()`</span> على الدالة.

</div>


In [ ]:
help(toa_radiation_wrapper)

<div dir="rtl">

### حساب إشعاع أعلى الغلاف الجوي (TOA)

</div>


<div dir="rtl">

كما في السابق، لحساب إشعاع <span dir="ltr">TOA</span> نحتاج إلى إنشاء شبكة خطوط عرض حتى نستطيع **توجيه/تَجْهِيز** (vectorize) خطوة <span dir="ltr">`toa_radiation()`</span>.

</div>


In [ ]:
lat_grid = ds_merra2.lat.values.reshape(1, 361, 1)\
    .repeat(ds_merra2.lon.size, axis = 2)\
    .repeat(ds_merra2.time.size, axis = 0)
lat_grid.shape

In [ ]:
ds_merra2['lat_grid'] = (('time', 'lat', 'lon'), lat_grid)
ds_merra2

<div dir="rtl">

ونحتاج أيضًا إلى **يوم السنة**، لكنه متاح مسبقًا كالتالي: <span dir="ltr">`ds_merra2['time.dayofyear']`</span>.

إذًا، نحن جاهزون لحساب إشعاع <span dir="ltr">TOA</span>!

</div>


In [ ]:
# Converting TOA Radiation from [MJ m-2 day-1] to [mm H2O day-1]
toa_rad = toa_radiation_wrapper(ds_merra2) * 0.408
toa_rad.sel(time = '2024-05-01').plot()

<div dir="rtl">

### حساب التبخر-نتح الكامن (PET)

</div>


<div dir="rtl">

والآن سنقوم أيضًا باقتطاع بيانات <span dir="ltr">MERRA-2</span> لمنطقة تيارت.

</div>


In [ ]:
toa_rad.shape

In [ ]:
import numpy as np

ds_merra2['toa_radiation'] = toa_rad

# Select just the Tiaret region
merra2_tiaret = ds_merra2.sel(lon = -1.32, lat = 35.37, method = 'nearest')

# Compute PET using the Hargreaves equation
merra2_tiaret_pet = 0.0023 * merra2_tiaret['toa_radiation'] * np.sqrt(
    merra2_tiaret['T2MMAX'] - merra2_tiaret['T2MMIN']) * (merra2_tiaret['T2MMEAN'] + 17.8)

<div dir="rtl">

### إعادة أخذ عينات بيانات CHIRPS

</div>


<div dir="rtl">

أخيرًا، نحتاج إلى معالجة بيانات الهطول الشهرية من <span dir="ltr">CHIRPS</span> وتحويلها إلى هطول يومي لمنطقة تيارت، كما فعلنا في الدرس السابق.

</div>


In [ ]:
chirps_tiaret = ds_chirps['precip'].sel(x = slice(0.8, 1.8), y = slice(36.1, 35.1))
chirps_tiaret_daily = chirps_tiaret.resample(time = 'D').nearest().mean(['x', 'y']) / 30
chirps_tiaret_daily

<div dir="rtl">

### اختيار جزء من سلسلة زمنية في `xarray`

الآن نحن جاهزون لمقارنة عامي <span dir="ltr">2023</span> و<span dir="ltr">2024</span>.

بافتراض أن مجلد <span dir="ltr">`data_raw/MERRA2`</span> يحتوي على البيانات التي نزّلناها لكل من <span dir="ltr">2023</span> و<span dir="ltr">2024</span>، فإن مجموعة البيانات <span dir="ltr">`ds_merra2`</span> ستحتوي على بيانات السنتين. رأينا كيف نختار تاريخًا محددًا باستخدام النمط <span dir="ltr">`sel(time = 'YYYY-MM-DD')`</span>، لكن كيف نختار كل البيانات الخاصة بسنة أو شهر أو نطاق تواريخ؟

كل بُعد في <span dir="ltr">`xarray`</span> يكون نوعه <span dir="ltr">`datetime64[ns]`</span> يمتلك مُلحق وصول زمني <span dir="ltr">DatetimeAccessor</span> عبر الخاصية <span dir="ltr">`dt`</span>.

</div>


In [ ]:
merra2_tiaret_pet.time.dt

<div dir="rtl">

<span dir="ltr">[The `dt` property can be used to subset a time series.](https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html)</span>

</div>


In [ ]:
merra2_tiaret_pet.time.dt.year

<div dir="rtl">

مثلًا، لاختيار البيانات الخاصة بعام <span dir="ltr">2023</span> فقط...

</div>


In [ ]:
# Select only those data points along the time access where time.year is in a list of years
merra2_tiaret_pet.sel(time = merra2_tiaret_pet.time.dt.year.isin([2023]))

In [ ]:
ratios = []
precip = []
for year in [2023, 2024]:
    pet_this_year = merra2_tiaret_pet.sel(time = merra2_tiaret_pet.time.dt.year.isin([year]))
    precip_this_year = chirps_tiaret_daily.sel(time = chirps_tiaret_daily.time.dt.year.isin([year]))
    # Select the first 122 days
    pet_this_year = pet_this_year.isel(time = slice(0, 122))
    precip_this_year = precip_this_year.isel(time = slice(0, 122))
    # NOTE: We must use the .values attribute because these are from two different datasets
    ratios.append(100 * precip_this_year.values / pet_this_year.values)
    precip.append(precip_this_year.values)

In [ ]:
pyplot.figure(figsize = (12, 5))
for y, year in enumerate([2023, 2024]):
    pyplot.plot(pet_this_year.time, ratios[y], label = year)
    
pyplot.ylabel('Precipitation-to-PET Ratio (mm day-1)')
pyplot.legend()

<div dir="rtl">

**يوحي الرسم أعلاه أن الظروف الهيدرولوجية في بداية <span dir="ltr">2024</span> ليست مختلفة كثيرًا عن العام السابق.** لكن لماذا أنتجت هذه الظروف الجافة موسميًا **جفافًا اجتماعيًا-اقتصاديًا** في <span dir="ltr">2024</span> ولم تنتجه في <span dir="ltr">2023</span>؟ كما رأينا عندما رسمنا نسب الهطول إلى <span dir="ltr">PET</span> على المدى الطويل باستخدام بيانات <span dir="ltr">TerraClimate</span>، فإن هطول موسم الرطوبة كان أقل من المعتاد لعدة سنوات. أحد التفسيرات الممكنة هو أن الأثر **التراكمي** لعدة سنوات من انخفاض الهطول أدى إلى ظروف جفاف في <span dir="ltr">2024</span> لم تكن واضحة بنفس الدرجة في <span dir="ltr">2023</span>.

يمكننا تصور الانخفاض طويل الأمد في هطول الشتاء مرة أخرى باستخدام بيانات <span dir="ltr">CHIRPS</span>. رغم وجود تحسن نسبي في <span dir="ltr">2022</span>، يبدو أن هطول الشتاء كان منخفضًا بشكل غير معتاد منذ <span dir="ltr">2019</span>.

</div>


In [ ]:
chirps_tiaret.sel(time = chirps_tiaret.time.dt.month.isin([1,2,3,4,5])).groupby('time.year').sum().mean(['x', 'y']).plot()
pyplot.ylabel('January - May Precipitation (mm)')

<div dir="rtl">

---

## موارد إضافية

- <span dir="ltr">[`xarray.core.accessor_dt.DatetimeAccessor` (Documentation)](https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html)</span>

</div>
